**Install Required Libraries**

In [25]:
import requests
from bs4 import BeautifulSoup

**Dependencies**

In [26]:
import pandas as pd
import numpy as np
import itertools
import re
import random
import schedule
import time

**Creating BS4 Functions for scrapping**

Movies are cateogirsed into seven and each category is processed by indivitual team members.

- category 1: 1940 to 1980 200 movie listing with rating=10000
- category 2: 2020 to 2021 200 movie listing with rating=20000
- category 3: 2000 to 2021 200 movie listing with rating=60000
- category 4: 1940 to 1980 200 movie listing with rating=10000
- category 5: 1980 to 2019 200 movie listing with rating=500000
- category 6: 1980 to 2019 200 movie listing with rating=80000
- category 7: 2005 to 2010 200 movie listing with rating=5000

## Sample files are put into Data_scrapped folder.

In [35]:
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2009-01-01,2011-12-31&num_votes=2000,&count=200" #2000 - 2021 6000votes filter 200 titles (imdb not letting to filter >200 titles in a go)

def getSoup(url):
    """
    Utility function this get soup function will fetch the above url which stored in url var.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def getReviews(soup):
    '''Function returns all reviews including positive and negative..'''

    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in soup.find_all('span', attrs={'class': 'point-scale'})]

    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    ans = []
    for i in range(5):
        ans.append(user_review_list[random.randint(0, len(user_review_list) - 1)])
    links = ["https://www.imdb.com" + tag['href'] for tag in ans]
    return links

def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    return tag.getText()

def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find h1 tag
    tag = soup.find('h1')
    return list(tag.children)[1].getText()

def scrape_reviews():
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2009-01-01,2011-12-31&num_votes=2000,&count=200"
    movies_soup = getSoup(url)

    movie_tags = movies_soup.find_all('a', attrs={'class': None})
    movie_tags = [tag.attrs['href'] for tag in movie_tags if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]
    movie_tags = list(dict.fromkeys(movie_tags))

    base_url = "https://www.imdb.com"
    movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
    movie_soups = [getSoup(link) for link in movie_links]

    movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]
    movie_review_list = list(itertools.chain(*movie_review_list))

    review_texts = [getReviewText(url) for url in movie_review_list]
    movie_titles = [getMovieTitle(url) for url in movie_review_list]

    df = pd.DataFrame({'user_review': review_texts})
    text_list = [m for m in df['user_review']]
    text_list_length = [len(m.split()) for m in text_list]
    df['length'] = text_list_length
    df = df[df['length'] < 250]
    df.drop('length', axis=1, inplace=True)
    df.to_csv('data_scrapped/data.csv', index=False)

    import csv
    with open("data_scrapped/data.csv", "r") as f:
        reader = csv.reader(f)
        rownumber = 2639
        for row in reader:
            g = open(f"data_scrapped/{rownumber}.txt", "w")
            g.write(str(row))
            rownumber += 1
            g.close()

def job():
    print("Starting the scraping job...")
    scrape_reviews()
    print("Scraping job finished.")

# Schedule the job every day at a specific time (e.g., 02:00 AM)
schedule.every().day.at("02:00").do(job)

print("Scheduler is running... Press Ctrl+C to stop.")


Scheduler is running... Press Ctrl+C to stop.
